In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.naive_bayes import MultinomialNB

In [43]:
data = pd.read_csv("/content/website_classification.csv")

In [44]:
data.head(5)

,Unnamed: 0,website_url,cleaned_website_text,Category
0,0,https://www.booking.com/index.html?aid=1743217,official site good hotel accommodation big sav...,Travel
1,1,https://travelsites.com/expedia/,expedia hotel book sites like use vacation wor...,Travel
2,2,https://travelsites.com/tripadvisor/,tripadvisor hotel book sites like previously d...,Travel
3,3,https://www.momondo.in/?ispredir=true,cheap flights search compare flights momondo f...,Travel
4,4,https://www.ebookers.com/?AFFCID=EBOOKERS-UK.n...,bot create free account create free account si...,Travel


In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1408 entries, 0 to 1407
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            1408 non-null   int64 
 1   website_url           1408 non-null   object
 2   cleaned_website_text  1408 non-null   object
 3   Category              1408 non-null   object
dtypes: int64(1), object(3)
memory usage: 44.1+ KB


In [46]:
data = data.drop_duplicates(subset='website_url').reset_index(drop=True)

In [47]:
data.Category.unique()

array(['Travel', 'Social Networking and Messaging', 'News',
       'Streaming Services', 'Sports', 'Photography',
       'Law and Government', 'Health and Fitness', 'Games', 'E-Commerce',
       'Forums', 'Food', 'Education', 'Computers and Technology',
       'Business/Corporate', 'Adult'], dtype=object)

In [48]:
data.Category = data.Category.replace({'Travel': 0, 'Social Networking and Messaging': 1, 'News': 2,
       'Streaming Services': 3, 'Sports': 4, 'Photography': 5,
       'Law and Government': 6, 'Health and Fitness': 7, 'Games': 8, 'E-Commerce': 9,
       'Forums': 10, 'Food': 11, 'Education': 12, 'Computers and Technology': 13,
       'Business/Corporate': 14, 'Adult': 15})

In [49]:
x = data.drop(labels = ['Category', 'Unnamed: 0', 'website_url'], axis = 1)
y = data.Category

In [60]:
x

<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'global_average_pooling1d_1')>

In [50]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [51]:
train_sentences[:10]

,cleaned_website_text
1260,data compression resource data compression res...
1156,online shopping site india shop electronics mo...
820,scrabble word finder dictionary cheat anagram ...
704,good hospital india multispeciality health car...
1097,burgerbae clothing vintage vibe week week new ...
1091,online fashion homeware kids clothes shopping ...
1237,tagspace versatile file organizer tagspaces op...
420,travel photography fashion photography documen...
503,page find sorry find page request follow follo...
471,pimp cam pimp cam kamera styling kamera kunst ...


In [52]:
train_labels

1260    13
1156     9
820      8
704      7
1097     9
        ..
1095     9
1130     9
1294    14
860     10
1126     9
Name: Category, Length: 1107, dtype: int64

In [53]:
train_sentences.shape, train_labels.shape

((1107, 1), (1107,))

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
baseline = Pipeline([
    ('changing into numbers', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

In [61]:
baseline.fit(train_sentences, train_labels)

ValueError: ignored

In [56]:
from tensorflow.keras.layers import TextVectorization
text_vectorizer = TextVectorization(max_tokens = 10000,
                                    output_mode = 'int',
                                    output_sequence_length = 15)
text_vectorizer.adapt(train_sentences)

In [57]:
embedding = tf.keras.layers.Embedding(input_dim = 10000,
                                      output_dim = 128,
                                      input_length = 15)

In [58]:
from tensorflow.keras import layers
import tensorflow as tf
inputs = layers.Input(shape = (1,), dtype = "string")
x = text_vectorizer(inputs)

x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_1 = tf.keras.Model(inputs, outputs)
model_1.compile(loss = 'categorical_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])
model_1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_1  (None, 128)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 1280129 (

In [59]:
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs = 10,
                              validation_data = (test_sentences, test_labels))

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


35/35 [==============================] - 2s 33ms/step - loss: 0.0000e+00 - accuracy: 0.0632 - val_loss: 0.0000e+00 - val_accuracy: 0.0578
Epoch 2/10


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


35/35 [==============================] - 1s 25ms/step - loss: 0.0000e+00 - accuracy: 0.0650 - val_loss: 0.0000e+00 - val_accuracy: 0.0578
Epoch 3/10
35/35 [==============================] - 1s 26ms/step - loss: 0.0000e+00 - accuracy: 0.0687 - val_loss: 0.0000e+00 - val_accuracy: 0.0578
Epoch 4/10
35/35 [==============================] - 1s 24ms/step - loss: 0.0000e+00 - accuracy: 0.0696 - val_loss: 0.0000e+00 - val_accuracy: 0.0578
Epoch 5/10
35/35 [==============================] - 1s 25ms/step - loss: 0.0000e+00 - accuracy: 0.0714 - val_loss: 0.0000e+00 - val_accuracy: 0.0578
Epoch 6/10
35/35 [==============================] - 1s 27ms/step - loss: 0.0000e+00 - accuracy: 0.0723 - val_loss: 0.0000e+00 - val_accuracy: 0.0578
Epoch 7/10
35/35 [==============================] - 1s 26ms/step - loss: 0.0000e+00 - accuracy: 0.0723 - val_loss: 0.0000e+00 - val_accuracy: 0.0578
Epoch 8/10
35/35 [==============================] - 1s 25ms/step - loss: 0.0000e+00 - accuracy: 0.0723 - val_loss: 0.